# Water Surface Elevation in 3D

In [ ]:
import xarray as xr

from schimpy import schism_mesh

import pandas as pd
import holoviews as hv
from holoviews import opts, dim

## Open output dataset containing the water surface elevations

In [ ]:
ds = xr.open_dataset('../tests/data/m1_hello_schism/outputs/out2d_1.nc')
ds

## Read mesh data of elements and nodes

This might not work for quad elements. Will have to deal with those as two non-overlapping triangles

In [ ]:
smesh = schism_mesh.read_mesh('../tests/data/m1_hello_schism/hgrid.gr3')

dfelems = pd.DataFrame(smesh.elems,columns=[0,1,2])
#dfelems

dfnodes = pd.DataFrame(smesh.nodes, columns=['x','y','z'])
dfnodes.z = -dfnodes.z
#dfnodes

## TriSurface plots from Plotly

Needed to add simplices from existing elements information of mesh rather than Delaunay triangulation of nodes

#Adapted from https://anaconda.org/philippjfr/brain/notebook?version=2017.05.04.1924

The code below allows for the simplices already defined by elems to be used instead of doing a Delaunay triangulation (used from scipy as a way to calculate the simplices)

In [ ]:
hv.extension('plotly')

import param

class TriSurface(hv.TriSurface):
    
    simplices = param.Array()

class TriSurfacePlot(hv.plotting.plotly.TriSurfacePlot):

    style_opts = ['cmap', 'plot_edges']

    def get_data(self, element, ranges, style, **kwargs):
        if element.simplices is None:
            return super(TriSurfacePlot, self).get_data(element, ranges, style, **kwargs)
        x, y, z = (element.dimension_values(i) for i in range(3))
        simplices = element.simplices
        return [dict(x=x, y=y, z=z, simplices=simplices)]
    
hv.Store.register({TriSurface: TriSurfacePlot}, 'plotly')

## Show mesh and water surface

The water surface simplices are derived from the nodes with Delaunay triangulation. Can we get the simplices from the information in the schism output files?

In [ ]:
dfelev = dfnodes.copy()

#ds.elevation.min(), ds.elevation.max()

mesh_surface = TriSurface(dfelev).opts(cmap='gray')

dfsurface = dfnodes.copy()

def show_surface(time=0):
    dfsurface.z = ds.elevation.values[time,:]
    #tris = TriSurface(dfelev, simplices = dfelems.values).opts(width=800, zlim=(0,2))
    #use Delaunay triangulation to build surface
    water_surface = hv.TriSurface(dfsurface).opts(width=800, zlim=(-10,2))
    return mesh_surface*water_surface.opts(plot_edges=False, cmap='kbc')

hv.output(widget_location='top')
dmap = hv.DynamicMap(show_surface,kdims=['time'], 
                     cache_size=1, ).opts(
    width=800, height=800)
dmap.redim.range(time=(0,len(ds.time)))